<a href="https://colab.research.google.com/github/Jeongwoo-KGI/sides-and-school/blob/main/suggestion_AI_food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This is code for recommend food if old person come to shop
#if new one come than it print a new person with no recommendation
import face_recognition
import cv2
import numpy as np
import pandas as pd
import time
#video_capture = cv2.VideoCapture('172.20.10.3:8080')
video_capture = cv2.VideoCapture(0)

# In this we read previous data and append the data like face encodings name previous orders
smp_dat=pd.read_csv('details_7.csv')
known_face_encodings=[]#stores face encoding
known_face_names=[]#store names 
food_order=[]# previous food order
for i in range(len(smp_dat)):
    img_name=str(smp_dat['mobileno'][i])
    path=img_name+".jpeg"
    image1 = face_recognition.load_image_file(path)
    encoding = face_recognition.face_encodings(image1)[0]

    known_face_encodings.append(encoding)
  
    known_face_names.append(smp_dat['name'])
    food_order.append(smp_dat['foodorder'][i])

#Now we detect the face if in databse
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
flag=0
flag_1=0
while True:
    ret, frame = video_capture.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]# BGR color(OpenCV) to RGB color (face_recognition)

    # Only process every other frame of video to save time
    if process_this_frame:
        # Get all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # Check if face matches from any saved images
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
                flag=1
            face_names.append(name)
    flag_1+=1
    #If person is not found database than break the loop 
    if flag==0:
        cv2.imshow('Video', frame)
        if flag_1==6:
            print("He is new person")
            break

    else:
        for (a,b,c,d), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            a *= 4
            b *= 4
            c *= 4
            d *= 4
            
            cv2.rectangle(frame, (d,a), (b, c), (0, 0, 255), 2)
        # Display the results    
        print("Hi ",name[best_match_index])
        print("Recommend the food")
        print(smp_dat['foodorder'][best_match_index])
        cv2.imshow('Video', frame)
        break
    if cv2.waitKey(5) & 0xFF == ord('q'):
        video_capture.release()
        cv2.destroyAllWindows()
        break
    
'''time.sleep(10)
video_capture.release()
cv2.destroyAllWindows()
'''

In [ ]:
import face_recognition
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array
import numpy as np
import cv2
import tensorflow.keras.backend as K
#import shlex e=shelx.split(file_read)
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
K.clear_session()
# Load the model
km = load_model(r'food_detect_model.hdf5',compile = False)

#video_capture = cv2.VideoCapture('172.20.10.3:8080')
video_capture = cv2.VideoCapture(0)
# e is denoted to the list of Food 101 items
e=['Apple pie', 'Baby back ribs', 'Baklava', 'Beef carpaccio', 'Beef tartare', 'Beet salad', 'Beignets', 'Bibimbap', 'Bread pudding', 'Breakfast burrito', 'Bruschetta', 'Caesar salad', 'Cannoli', 'Caprese salad', 'Carrot cake', 'Ceviche', 'Cheesecake', 'Cheese plate', 'Chicken curry', 'Chicken quesadilla', 'Chicken wings', 'Chocolate cake', 'Chocolate mousse', 'Churros', 'Clam chowder', 'Club sandwich', 'Crab cakes', 'Creme brulee', 'Croque madame', 'Cup cakes', 'Deviled eggs', 'Donuts', 'Dumplings', 'Edamame', 'Eggs benedict', 'Escargots', 'Falafel', 'Filet mignon', 'Fish and chips', 'Foie gras', 'French fries', 'French onion soup', 'French toast', 'Fried calamari', 'Fried rice', 'Frozen yogurt', 'Garlic bread', 'Gnocchi', 'Greek salad', 'Grilled cheese sandwich', 'Grilled salmon', 'Guacamole', 'Gyoza', 'Hamburger', 'Hot and sour soup', 'Hot dog', 'Huevos rancheros', 'Hummus', 'Ice cream', 'Lasagna', 'Lobster bisque', 'Lobster roll sandwich', 'Macaroni and cheese', 'Macarons', 'Miso soup', 'Mussels', 'Nachos', 'Omelette', 'Onion rings', 'Oysters', 'Pad thai', 'Paella', 'Pancakes', 'Panna cotta', 'Peking duck', 'Pho', 'Pizza', 'Pork chop', 'Poutine', 'Prime rib', 'Pulled pork sandwich', 'Ramen', 'Ravioli', 'Red velvet cake', 'Risotto', 'Samosa', 'Sashimi', 'Scallops', 'Seaweed salad', 'Shrimp and grits', 'Spaghetti bolognese', 'Spaghetti carbonara', 'Spring rolls', 'Steak', 'Strawberry shortcake', 'Sushi', 'Tacos', 'Takoyaki', 'Tiramisu', 'Tuna tartare', 'Waffles']
# In this we read previous data and append the data like face encodings name previous orders

smp_dat=pd.read_csv('details_7.csv')
known_face_encodings=[]#stores face encoding
known_face_names=[]#store names 
food_order=[]# previous food order
for i in range(len(smp_dat)):
    img_name=str(smp_dat['mobileno'][i])
    path=img_name+".jpeg"
    image1 = face_recognition.load_image_file(path)
    encoding = face_recognition.face_encodings(image1)[0]

    known_face_encodings.append(encoding)
  
    known_face_names.append(smp_dat['name'])
    food_order.append(smp_dat['foodorder'][i])
#Now we detect the face if in databse
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
flag=0
flag_1=0
while True:
    ret, frame = video_capture.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]# BGR color(OpenCV) to RGB color (face_recognition)

    # Only process every other frame of video to save time
    if process_this_frame:
        # Get all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # Check if face matches from any saved images
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
                flag=1

            face_names.append(name)

    flag_1+=1
    #If person is not found database than break the loop 
    if flag==0:
       cv2.imshow("person",frame)
       if flag_1==6:
          print("he is a new preson")
          break
   
    # Display the results
    else:
        for (a,b,c,d),name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            a *= 4
            b *= 4
            c *= 4
            d *= 4
         
            
            cv2.rectangle(frame, (d,a), (b, c), (0, 0, 0), 170)
            
        cv2.imshow('Video', frame)
        # Now we detect food before that we alerady removed the face 
        roi=cv2.resize(frame,(299, 299))
        roi=img_to_array(roi)
        roi=np.expand_dims(roi,axis=0)
        roi /= 255.       
        pred=km.predict(roi)
        #print(pred)
        max_e=max(pred)
        ind=pred.argmax()
        pred_value=e[ind]
        print(e[ind])
        #add the food in according to customer in database
        smp_dat['foodorder'][best_match_index]=food_order[best_match_index]+","+pred_value
        smp_dat.to_csv('details_7.csv',index=False)
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#If the person in new in database than it crate a new database
if flag==0:
    video_capture.release()
    v=cv2.VideoCapture(0)
    fd=cv2.CascadeClassifier(r'haarcascade_frontalface_alt2.xml')

    mobile_no=input("enter ur mobile no.")
    
    def cap():
        flag=1
        while(flag):
            ret,i1=v.read()
            j=cv2.cvtColor(i1,cv2.COLOR_BGR2GRAY)#for gray color detection
            f=fd.detectMultiScale(j)
      
            if len(f)==1:
                for x,y,w,h in f:
                    images=i1[y:y+h,x:x+w]#crop image
                    flag=0
          
                    cv2.imshow('image',images)
                    
                   
                    
                    

        return images
    i_1=cap()
    img_nam=mobile_no+".jpeg"
    cv2.imwrite(img_nam,i_1)
    name=input("enter name")
    
    food_given=input("What food you like to order")
    
        
    df=pd.read_csv('details_7.csv')
    print(df)
     
    #mmake data frame with columns
    df_1 = pd.DataFrame({'mobileno':[mobile_no],
                      'name': [name],
                      'foodorder':[food_given]
                        })

    #append it with previous dataset
    datafram=df.append(df_1, ignore_index=True)
    #save it in dataset
    datafram.to_csv('details_7.csv',index=False)

In [ ]:
#Before start This program in only work when web cam is good
# This only predict the food correctly if the frame of food u showing is greater than the ur face that displaying in background

from keras.preprocessing.image import img_to_array
import numpy as np
import cv2

import tensorflow.keras.backend as K
import shlex
from tensorflow.keras.models import load_model
import pandas as pd
K.clear_session()
km = load_model(r'food_detect_model.hdf5',compile = False)
#km=load_model(r'C:\Users\dheer\Desktop\food101_model\best_model_11class.hdf5',compile=False)
'''
e=shelx.split(file_read)'''
#e=['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_soup', 'mussels', 'nachos', 'omelette', 'onion_rings', 'oysters', 'pad_thai', 'paella', 'pancakes', 'panna_cotta', 'peking_duck', 'pho', 'pizza', 'pork_chop', 'poutine', 'prime_rib', 'pulled_pork_sandwich', 'ramen', 'ravioli', 'red_velvet_cake', 'risotto', 'samosa', 'sashimi', 'scallops', 'seaweed_salad', 'shrimp_and_grits', 'spaghetti_bolognese', 'spaghetti_carbonara', 'spring_rolls', 'steak', 'strawberry_shortcake', 'sushi', 'tacos', 'takoyaki', 'tiramisu', 'tuna_tartare', 'waffles']
df=pd.read_csv('calorie_data.csv')
#e=['Apple pie', 'Baby back ribs', 'Baklava', 'Beef carpaccio', 'Beef tartare', 'Beet salad', 'Beignets', 'Bibimbap', 'Bread pudding', 'Breakfast burrito', 'Bruschetta', 'Caesar salad', 'Cannoli', 'Caprese salad', 'Carrot cake', 'Ceviche', 'Cheesecake', 'Cheese plate', 'Chicken curry', 'Chicken quesadilla', 'Chicken wings', 'Chocolate cake', 'Chocolate mousse', 'Churros', 'Clam chowder', 'Club sandwich', 'Crab cakes', 'Creme brulee', 'Croque madame', 'Cup cakes', 'Deviled eggs', 'Donuts', 'Dumplings', 'Edamame', 'Eggs benedict', 'Escargots', 'Falafel', 'Filet mignon', 'Fish and chips', 'Foie gras', 'French fries', 'French onion soup', 'French toast', 'Fried calamari', 'Fried rice', 'Frozen yogurt', 'Garlic bread', 'Gnocchi', 'Greek salad', 'Grilled cheese sandwich', 'Grilled salmon', 'Guacamole', 'Gyoza', 'Hamburger', 'Hot and sour soup', 'Hot dog', 'Huevos rancheros', 'Hummus', 'Ice cream', 'Lasagna', 'Lobster bisque', 'Lobster roll sandwich', 'Macaroni and cheese', 'Macarons', 'Miso soup', 'Mussels', 'Nachos', 'Omelette', 'Onion rings', 'Oysters', 'Pad thai', 'Paella', 'Pancakes', 'Panna cotta', 'Peking duck', 'Pho', 'Pizza', 'Pork chop', 'Poutine', 'Prime rib', 'Pulled pork sandwich', 'Ramen', 'Ravioli', 'Red velvet cake', 'Risotto', 'Samosa', 'Sashimi', 'Scallops', 'Seaweed salad', 'Shrimp and grits', 'Spaghetti bolognese', 'Spaghetti carbonara', 'Spring rolls', 'Steak', 'Strawberry shortcake', 'Sushi', 'Tacos', 'Takoyaki', 'Tiramisu', 'Tuna tartare', 'Waffles']
e=list(df['categories'].values)

v=cv2.VideoCapture(0)
while(1):
   
    ret,im_1=v.read()
    
    roi=cv2.resize(im_1,(299, 299))
    roi=img_to_array(roi)
    roi=np.expand_dims(roi,axis=0)
    roi /= 255.       
    pred=km.predict(roi)
    cv2.imshow("f",im_1)
    max_e=max(pred)
    ind=pred.argmax()
    pred_value=e[ind]
    print(e[ind])

    print('Calories of food are',end=' ')
    print(df[df['categories']==pred_value ]['calories'].values[0])

    print("per weight in grams",end=' ')
    print(df[df['categories']== pred_value]['weight'].values[0])
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

In [ ]:
#This only work with image and food frame should be bigger than any object around it
from keras.preprocessing.image import img_to_array
import numpy as np
import cv2
import tensorflow.keras.backend as K
#import shlex e=shelx.split(file_read)
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



K.clear_session()
# Load the model
km = load_model(r'food_detect_model.hdf5',compile = False)
#Read the calorie and weight dataset 
df=pd.read_csv('calorie_data.csv')
# e is denoted to the list of Food 101 items
#e=['Apple pie', 'Baby back ribs', 'Baklava', 'Beef carpaccio', 'Beef tartare', 'Beet salad', 'Beignets', 'Bibimbap', 'Bread pudding', 'Breakfast burrito', 'Bruschetta', 'Caesar salad', 'Cannoli', 'Caprese salad', 'Carrot cake', 'Ceviche', 'Cheesecake', 'Cheese plate', 'Chicken curry', 'Chicken quesadilla', 'Chicken wings', 'Chocolate cake', 'Chocolate mousse', 'Churros', 'Clam chowder', 'Club sandwich', 'Crab cakes', 'Creme brulee', 'Croque madame', 'Cup cakes', 'Deviled eggs', 'Donuts', 'Dumplings', 'Edamame', 'Eggs benedict', 'Escargots', 'Falafel', 'Filet mignon', 'Fish and chips', 'Foie gras', 'French fries', 'French onion soup', 'French toast', 'Fried calamari', 'Fried rice', 'Frozen yogurt', 'Garlic bread', 'Gnocchi', 'Greek salad', 'Grilled cheese sandwich', 'Grilled salmon', 'Guacamole', 'Gyoza', 'Hamburger', 'Hot and sour soup', 'Hot dog', 'Huevos rancheros', 'Hummus', 'Ice cream', 'Lasagna', 'Lobster bisque', 'Lobster roll sandwich', 'Macaroni and cheese', 'Macarons', 'Miso soup', 'Mussels', 'Nachos', 'Omelette', 'Onion rings', 'Oysters', 'Pad thai', 'Paella', 'Pancakes', 'Panna cotta', 'Peking duck', 'Pho', 'Pizza', 'Pork chop', 'Poutine', 'Prime rib', 'Pulled pork sandwich', 'Ramen', 'Ravioli', 'Red velvet cake', 'Risotto', 'Samosa', 'Sashimi', 'Scallops', 'Seaweed salad', 'Shrimp and grits', 'Spaghetti bolognese', 'Spaghetti carbonara', 'Spring rolls', 'Steak', 'Strawberry shortcake', 'Sushi', 'Tacos', 'Takoyaki', 'Tiramisu', 'Tuna tartare', 'Waffles']

e=list(df['categories'].values)

im_1=cv2.imread(r'burger_1.jpg')
#remove face from image 
gray = cv2.cvtColor(im_1, cv2.COLOR_BGR2GRAY)
faceCascade = cv2.CascadeClassifier(r"haarcascade_frontalface_default.xml")
fd = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.3,
    minNeighbors=3,
    minSize=(30, 30)
)
for (x, y, w, h) in fd:
    cv2.rectangle(im_1, (x, y), (x + w, y + h), (0, 0, 0), 2)
    im_1[y:y+h,x:x+w]=0
#Convert the image and perform operation so every value of array comes in 0 to 1 

roi=cv2.resize(im_1,(299, 299))
roi=img_to_array(roi)
roi=np.expand_dims(roi,axis=0)
roi /= 255.
#predict the food by the help of model
pred=km.predict(roi)
#print(pred)
max_e=max(pred)
ind=pred.argmax()
pred_value=e[ind]
print(e[ind])

#Calculating calories and weight of food
print('Calories of food are',end=' ')
print(df[df['categories']==pred_value ]['calories'].values[0])

print("per weight in grams",end=' ')
print(df[df['categories']== pred_value]['weight'].values[0])
#Now we sugest the food to person by calculating obesity level form height,weight and gender
calo_per_wght=df[df['categories']== pred_value]['cal_per_weight'].values[0]
height=int(input("enter ur height"))
weight= int(input("enter ur weight")) 
gender= input("enter ur gender M for Male and F for Female")
if gender=='M':
           gender_no=0
else:
           gender_no=1
#making datframe
d = {'Height': [height], 'Weight': [weight],'Gender_no':[gender_no]}
df1 = pd.DataFrame(data=d)
#read the datset and split into test and train
data=pd.read_csv('bmi_level.csv')
labels=pd.DataFrame(data['Index'])
features=data.drop(['Gender','Index'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=0.1,random_state=42)
clf=SVC(kernel='linear')
clf.fit(features,labels)

y_pred=clf.predict(df1)
#After prediction of obesity level we suggest the food to person
if y_pred[0] ==4 or y_pred[0] ==5:
    if calo_per_wght >2.5 :
           print("This food is of high calories so We don't recommended that")
    else :
           print("You can eat that food It is of low calories")
elif y_pred[0] ==3:
    if calo_per_wght >3.5 :
           print("This food is of high calories so We don't recommended that")
    else :
           print("You can eat that food It is of low calories")

elif y_pred[0] ==2:
    if calo_per_wght >4.5 :
           print("This food is of high calories so We don't recommended that")
    else :
           print("You can eat that food It is of low calories")

elif y_pred[0] ==1:
    print("You can eat that food")
           